# Домашнее задание №5. Autoencoder

In [ ]:
import os
import math
import json

from IPython.display import display, clear_output
from scipy.sparse import csr_matrix

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Подготовка данных

In [ ]:
DATA_PATH = "/content/drive/MyDrive/recsys_course/data_kion"

In [ ]:
interactions_df = pd.read_csv(f'{DATA_PATH}/interactions_processed_kion.csv')
users_df = pd.read_csv(f'{DATA_PATH}/users_processed_kion.csv')
items_df = pd.read_csv(f'{DATA_PATH}/items_processed_kion.csv')

In [ ]:
interactions_df.head()

,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,176549,9506,2021-05-11,4250,72
1,699317,1659,2021-05-29,8317,100
2,656683,7107,2021-05-09,10,0
3,864613,7638,2021-07-05,14483,100
4,964868,9506,2021-04-30,6725,100


In [ ]:
interactions_df = interactions_df[interactions_df['last_watch_dt'] < '2021-04-01']

In [ ]:
interactions_df.shape

(263874, 5)

In [ ]:
# оставляем пользователей, у которых есть хотя бы 5 просмотров
users_interactions_count_df = interactions_df.groupby(['user_id', 'item_id']).size().groupby('user_id').size()
print('# users: %d' % len(users_interactions_count_df))
users_with_enough_interactions_df = users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['user_id']]
print('# users with at least 5 interactions: %d' % len(users_with_enough_interactions_df))

# users: 86614
# users with at least 5 interactions: 14563


In [ ]:
print('# of interactions: %d' % len(interactions_df))
interactions_from_selected_users_df = interactions_df.merge(users_with_enough_interactions_df,
               how = 'right',
               left_on = 'user_id',
               right_on = 'user_id')
print('# of interactions from users with at least 5 interactions: %d' % len(interactions_from_selected_users_df))

# of interactions: 263874
# of interactions from users with at least 5 interactions: 142670


In [ ]:
# сглаживаем веса (процент просмотра)
def smooth_user_preference(x):
    return math.log(1+x, 2)

interactions_full_df = interactions_from_selected_users_df \
                    .groupby(['user_id', 'item_id'])['watched_pct'].sum() \
                    .apply(smooth_user_preference).reset_index()
print('# of unique user/item interactions: %d' % len(interactions_full_df))
interactions_full_df.head(10)

# of unique user/item interactions: 142670


,user_id,item_id,watched_pct
0,21,849,6.375039
1,21,4345,6.658211
2,21,10283,6.658211
3,21,12261,6.658211
4,21,15997,6.658211
5,32,952,6.044394
6,32,4382,4.954196
7,32,4807,6.658211
8,32,10436,6.658211
9,32,12132,6.658211


### Разбиение данных на трейн и тест

In [ ]:
interactions_train_df, interactions_test_df = train_test_split(interactions_full_df,
                                   stratify=interactions_full_df['user_id'],
                                   test_size=0.20,
                                   random_state=1008)

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

# interactions on Train set: 114136
# interactions on Test set: 28534


In [ ]:
#Indexing by personId to speed up the searches during evaluation
interactions_full_indexed_df = interactions_full_df.set_index('user_id')
interactions_train_indexed_df = interactions_train_df.set_index('user_id')
interactions_test_indexed_df = interactions_test_df.set_index('user_id')

In [ ]:
def get_items_interacted(person_id, interactions_df):
    # Get the user's data and merge in the movie information.
    interacted_items = interactions_df.loc[person_id]['item_id']
    return set(interacted_items if type(interacted_items) == pd.Series else [interacted_items])

## Обучение

### Преобразование данных для обучения

In [ ]:
# Constants
SEED = 1008 # random seed for reproducibility
LR = 1e-3 # learning rate, controls the speed of the training
WEIGHT_DECAY = 0.01 # lambda for L2 reg.
NUM_EPOCHS = 200 # num training epochs (how many times each instance will be processed)
GAMMA = 0.9995 # learning rate scheduler parameter
BATCH_SIZE = 3000 # training batch size
EVAL_BATCH_SIZE = 3000 # evaluation batch size.
DEVICE = (
    "cuda" if torch.cuda.is_available() else "cpu"
) #'cuda' # device to make the calculations on

In [ ]:
total_df = interactions_train_df.append(interactions_test_indexed_df.reset_index())
total_df['user_id'], users_keys = total_df.user_id.factorize()
total_df['item_id'], items_keys = total_df.item_id.factorize()

train_encoded = total_df.iloc[:len(interactions_train_df)].values
test_encoded = total_df.iloc[len(interactions_train_df):].values

<ipython-input-15-50c21e885cbc>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_df = interactions_train_df.append(interactions_test_indexed_df.reset_index())


In [ ]:
shape = [int(total_df['user_id'].max()+1), int(total_df['item_id'].max()+1)]
X_train = csr_matrix((train_encoded[:, 2], (train_encoded[:, 0], train_encoded[:, 1])), shape=shape).toarray()
X_test = csr_matrix((test_encoded[:, 2], (test_encoded[:, 0], test_encoded[:, 1])), shape=shape).toarray()

In [ ]:
# Initialize the DataObject, which must return an element (features vector x and target value y)
# for a given idx. This class must also have a length atribute
class UserOrientedDataset(Dataset):
    def __init__(self, X):
        super().__init__() # to initialize the parent class
        self.X = X.astype(np.float32)
        self.len = len(X)

    def __len__(self): # We use __func__ for implementing in-built python functions
        return self.len

    def __getitem__(self, index):
        return self.X[index]

In [ ]:
# Initialize DataLoaders - objects, which sample instances from DataObject-s
train_dl = DataLoader(
    UserOrientedDataset(X_train),
    batch_size = BATCH_SIZE,
    shuffle = True
)

test_dl = DataLoader(
    UserOrientedDataset(X_test),
    batch_size = EVAL_BATCH_SIZE,
    shuffle = False
)

dls = {'train': train_dl, 'test': test_dl}

### Модель

In [ ]:
class Model(nn.Module):
    def __init__(self, in_and_out_features = 8287):
        super().__init__()
        self.in_and_out_features = in_and_out_features

        self.sequential = nn.Sequential( # NN architecure, where the modules modify the data sequentially
            nn.Linear(in_and_out_features, 512), # Linear transformation
            nn.ReLU(), # Activation function
            nn.Linear(512, 128), # Linear transformation
            nn.ReLU(), # Activation function
            nn.Linear(128, 64), # Linear transformation
            nn.ReLU(), # Activation function
            nn.Linear(64, 32), # Linear transformation
            nn.ReLU(), # Activation function
            nn.Linear(32, 64), # Linear transformation
            nn.ReLU(), # Activation function
            nn.Linear(64, 128), # Linear transformation
            nn.ReLU(), # Activation function
            nn.Linear(128, 512), # Linear transformation
            nn.ReLU(), # Activation function
            nn.Linear(512, in_and_out_features) # Another Linear transformation
        )

    def forward(self, x): # In the forward function, you define how your model runs, from input to output
        x = self.sequential(x)
        return x

In [ ]:
torch.manual_seed(SEED) # Fix random seed to have reproducible weights of model layers

model = Model()
model.to(DEVICE)

# Initialize GD method, which will update the weights of the model
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
# Initialize learning rate scheduler, which will decrease LR according to some rule
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=GAMMA)

def rmse_for_sparse(x_pred, x_true):
    mask = (x_true > 0)
    sq_diff = (x_pred * mask - x_true) ** 2
    mse = sq_diff.sum() / mask.sum()
    return mse ** (1/2)

In [ ]:
# Training loop
metrics_dict = {
    "Epoch": [],
    "Train RMSE": [],
    "Test RMSE": [],
}

# Train loop
for epoch in range(NUM_EPOCHS):
    metrics_dict["Epoch"].append(epoch)
    for stage in ['train', 'test']:
        with torch.set_grad_enabled(stage == 'train'): # Whether to start building a graph for a backward pass
            if stage == 'train':
                model.train() # Enable some "special" layers (will speak about later)
            else:
                model.eval() # Disable some "special" layers (will speak about later)

            loss_at_stage = 0
            for batch in dls[stage]:
                batch = batch.to(DEVICE)
                x_pred = model(batch) # forward pass: model(x_batch) -> calls forward()
                loss = rmse_for_sparse(x_pred, batch) # ¡Important! y_pred is always the first arg
                if stage == "train":
                    loss.backward() # Calculate the gradients of all the parameters wrt loss
                    optimizer.step() # Update the parameters
                    scheduler.step()
                    optimizer.zero_grad() # Zero the saved gradient
                loss_at_stage += loss.item() * len(batch)
            rmse_at_stage = (loss_at_stage / len(dls[stage].dataset)) ** (1/2)
            metrics_dict[f"{stage.title()} RMSE"].append(rmse_at_stage)

    if (epoch == NUM_EPOCHS - 1) or epoch % 10 == 9:
        clear_output(wait=True)
        display(pd.DataFrame(metrics_dict))

,Epoch,Train RMSE,Test RMSE
0,0,2.329693,2.316036
1,1,2.298008,2.270677
2,2,2.104916,2.055495
3,3,1.972623,2.041320
4,4,1.781872,2.027229
...,...,...,...
195,195,0.794911,1.472577
196,196,0.795662,1.474937
197,197,0.793926,1.468267
198,198,0.794802,1.467567


## Метрики

In [ ]:
with torch.no_grad():
    X_pred = model(torch.Tensor(X_test).to(DEVICE))
X_pred

tensor([[ 1.4376,  1.0888,  0.8516,  ...,  0.0758, -0.1622, -0.6394],
        [ 3.9171,  3.0723,  1.5168,  ...,  0.1476,  0.8216, -0.9718],
        [ 5.6713,  1.7836,  2.5258,  ..., -0.5055, -0.2768, -0.5200],
        ...,
        [ 1.3314,  1.8022,  1.3678,  ..., -0.0326, -0.0302, -0.3867],
        [ 3.6214,  3.6640,  2.1495,  ..., -0.1458, -0.4913, -0.4825],
        [ 3.1083,  3.3853,  2.2843,  ..., -0.4502,  0.0456, -0.3540]],
       device='cuda:0')

In [ ]:
class AERecommender:

    MODEL_NAME = 'Autoencoder'

    def __init__(self, X_preds, X_train_and_val, X_test):

        self.X_preds = X_preds.cpu().detach().numpy()
        self.X_train_and_val = X_train_and_val
        self.X_test = X_test

    def get_model_name(self):
        return self.MODEL_NAME

    def recommend_items(self, user_id, items_to_select_idx, topn=10, verbose=False):
        user_preds = self.X_preds[user_id][items_to_select_idx]
        items_idx = items_to_select_idx[np.argsort(-user_preds)[:topn]]

        # Recommend the highest predicted rating movies that the user hasn't seen yet.
        return items_idx

    def evaluate(self, size=100):

        X_total = self.X_train_and_val + self.X_test

        true_5 = []
        true_10 = []

        for user_id in range(len(X_test)):
            non_zero = np.argwhere(self.X_test[user_id] > 0).ravel()
            all_nonzero = np.argwhere(X_total[user_id] > 0).ravel()
            select_from = np.setdiff1d(np.arange(X_total.shape[1]), all_nonzero)

            for non_zero_idx in non_zero:
                random_non_interacted_100_items = np.random.choice(select_from, size=20, replace=False)
                preds = self.recommend_items(user_id, np.append(random_non_interacted_100_items, non_zero_idx), topn=10)
                true_5.append(non_zero_idx in preds[:5])
                true_10.append(non_zero_idx in preds)

        return {"recall@5": np.mean(true_5), "recall@10": np.mean(true_10)}

ae_recommender_model = AERecommender(X_pred, X_train, X_train)

In [ ]:
ae_global_metrics = ae_recommender_model.evaluate()
ae_global_metrics

{'recall@5': 0.25805035557159717, 'recall@10': 0.5505457950854055}

## Получение предсказаний для сервиса

In [ ]:
total_encoded = total_df.values
shape = [int(total_df["user_id"].max() + 1), int(total_df["item_id"].max() + 1)]
X = csr_matrix(
    (total_encoded[:, 2], (total_encoded[:, 0], total_encoded[:, 1])), shape=shape
).toarray()

In [ ]:
with torch.no_grad():
    X_pred = model(torch.Tensor(torch.Tensor(X).to(DEVICE)))
X_pred

tensor([[ 4.4722,  5.8773,  4.3154,  ...,  0.5996,  0.6131, -0.5149],
        [ 5.8697,  5.3897,  5.8946,  ...,  0.7325,  1.8375, -0.2583],
        [ 5.1956,  6.0324,  5.5164,  ...,  0.1091,  1.9728, -0.6366],
        ...,
        [ 3.8789,  4.7260,  3.7442,  ...,  0.5862,  1.0569, -0.3499],
        [ 3.8643,  5.7387,  2.7461,  ..., -0.0816, -0.4153, -0.5403],
        [ 6.7303,  6.3493,  3.2696,  ...,  0.6466,  1.5299, -0.7070]],
       device='cuda:0')

In [ ]:
X_pred = X_pred.to('cpu')

In [ ]:
def recommend(user_id):
  all_nonzero = np.argwhere(X[user_id] > 0).ravel()
  select_from = np.setdiff1d(np.arange(X.shape[1]), all_nonzero)
  random_non_interacted_100_items = np.random.choice(select_from, size=20, replace=False)
  user_preds = X_pred[user_id][random_non_interacted_100_items]
  items_idx = random_non_interacted_100_items[np.argsort(-user_preds)[:10]]
  return items_idx

In [ ]:
recos = {}
users = interactions_full_df['user_id'].unique().tolist()
for i, user_id in enumerate(users):
    recos_for_user = recommend(i)
    recos.update({user_id: recos_for_user.tolist()})

In [ ]:
# сохраним рекомендации
RECOS_PATH = "/content/drive/MyDrive/recsys_course/recommendations"

with open(f"{RECOS_PATH}/autoencoder.json", "w") as f:
    json.dump(recos, f)